<a href="https://colab.research.google.com/github/Souravkumar2434/pytorch_framework/blob/main/ANN_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import torch
import numpy as np

In [3]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [5]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [6]:
df = df.drop(['id', 'Unnamed: 32'], axis = 1)

In [7]:
df.columns

Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')

In [8]:
X = df.iloc[:, 1:]
Y = df.diagnosis

In [9]:
X.shape, Y.shape

((569, 30), (569,))

In [10]:
label_encoding = LabelEncoder()
Y = label_encoding.fit_transform(Y)

In [11]:
Y[:5]

array([1, 1, 1, 1, 1])

In [12]:
scalar = StandardScaler()
X = scalar.fit_transform(X)

In [13]:
X[:1]

array([[ 1.09706398, -2.07333501,  1.26993369,  0.9843749 ,  1.56846633,
         3.28351467,  2.65287398,  2.53247522,  2.21751501,  2.25574689,
         2.48973393, -0.56526506,  2.83303087,  2.48757756, -0.21400165,
         1.31686157,  0.72402616,  0.66081994,  1.14875667,  0.90708308,
         1.88668963, -1.35929347,  2.30360062,  2.00123749,  1.30768627,
         2.61666502,  2.10952635,  2.29607613,  2.75062224,  1.93701461]])

In [14]:
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

In [16]:
X_train.shape, Y_train.shape

((455, 30), (455,))

In [17]:
X_test.shape, Y_test.shape

((114, 30), (114,))

In [18]:
class CustomDataset(Dataset):
  def __init__(self, features, labels):
    self.features = features
    self.labels = labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
    return self.features[index], self.labels[index]



In [29]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
Y_train_tensor = torch.from_numpy(Y_train.astype(np.float32))
Y_test_tensor = torch.from_numpy(Y_test.astype(np.float32))

In [30]:
train_dataset = CustomDataset(X_train_tensor, Y_train_tensor)

In [31]:
test_dataset = CustomDataset(X_test_tensor, Y_test_tensor)

In [32]:
train_dataset[0]

(tensor([-0.6496, -0.1372, -0.5783, -0.6095,  1.0347,  0.8956,  0.4143,  0.0748,
          1.7867,  2.1537,  0.2791, -0.3390,  0.3014,  0.0146, -0.4967,  0.4841,
          0.3368, -0.2200,  0.2649,  0.7081,  0.1141,  0.3978,  0.3616,  0.0143,
          1.3734,  2.0562,  2.0313,  0.6090,  3.0095,  3.1174]),
 tensor(1.))

In [33]:
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size= 32, shuffle = True)

In [34]:
# Defining the model

class ANN(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):
    out = self.linear(features)
    out = self.sigmoid(out)
    return out

In [35]:
# defining the parameters

learning_rate = 0.1
epochs = 50

In [36]:
# Creating model

model = ANN(X_train_tensor.shape[1])

# optimizer
optim = torch.optim.SGD(model.parameters(), lr = learning_rate)

loss_function = nn.BCELoss()

In [38]:
# Pipeline

for epoch in range(epochs):
  for batch_features, batch_labels in train_loader:
    # Forward pass
    y_pred = model(batch_features)
    # loss calculation
    loss = loss_function(y_pred, batch_labels.view(-1, 1))
    # clear gradients
    optim.zero_grad()
    # babck prpogation
    loss.backward()
  print(f"Epoch {epoch + 1}, loss: {loss.item()}")




Epoch 1, loss: 1.0576322078704834
Epoch 2, loss: 0.7844699025154114
Epoch 3, loss: 1.062072515487671
Epoch 4, loss: 0.8775420784950256
Epoch 5, loss: 0.7703313231468201
Epoch 6, loss: 1.0040745735168457
Epoch 7, loss: 0.8720674514770508
Epoch 8, loss: 0.8830078840255737
Epoch 9, loss: 0.9945061802864075
Epoch 10, loss: 1.2399572134017944
Epoch 11, loss: 1.0493600368499756
Epoch 12, loss: 0.7512105107307434
Epoch 13, loss: 0.9570848345756531
Epoch 14, loss: 0.9103749394416809
Epoch 15, loss: 0.772075355052948
Epoch 16, loss: 0.8401508331298828
Epoch 17, loss: 0.729216456413269
Epoch 18, loss: 0.777251660823822
Epoch 19, loss: 0.8965867757797241
Epoch 20, loss: 0.7727168202400208
Epoch 21, loss: 0.8505644798278809
Epoch 22, loss: 0.8788232803344727
Epoch 23, loss: 0.6328242421150208
Epoch 24, loss: 0.7359287142753601
Epoch 25, loss: 0.9763911366462708
Epoch 26, loss: 0.7158594727516174
Epoch 27, loss: 0.9559035301208496
Epoch 28, loss: 1.2329362630844116
Epoch 29, loss: 1.024110436439514

In [40]:
# Model Evaluation

model.eval() # Setting the model in the evaluation mode

accuracy_list = []

with torch.no_grad():
  for batch_features, batch_labels in test_loader:
    # forward pass
    y_pred = model(batch_features)
    y_pred = (y_pred > 0.8).float()

    batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
    accuracy_list.append(batch_accuracy)

overall_Accuracy = sum(accuracy_list) / len(accuracy_list)

print(f"Accuracy: {overall_Accuracy:.4f}")



Accuracy: 0.6519
